# Ray Tasks Revisited

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

The [Ray Crash Course](../ray-crash-course/00-Ray-Crash-Course-Overview.ipynb) introduced the core concepts of Ray's API and how they parallelize work. Specifically, we learned how to define Ray _tasks_ and _actors_, run them, and retrieve the results. 

This lesson explores Ray tasks in greater depth, including the following:

* How task dependencies are handled automatically by Ray
* Usage patterns for `ray.get()` and `ray.wait()`
* Specifying limits on the number of invocations and retries on failure
* An exploration of task granularity considerations
* Profiling tasks

In [1]:
import ray, time, os, sys 
import numpy as np 
sys.path.append("..")
from util.printing import pd, pnd  # convenience methods for printing results.

In [2]:
ray.init(ignore_reinit_error=True)

2022-03-16 15:54:23,385	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-16_15-54-20_987355_57958/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-16_15-54-20_987355_57958/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-16_15-54-20_987355_57958',
 'metrics_export_port': 55497,
 'gcs_address': '127.0.0.1:60165',
 'address': '127.0.0.1:60165',
 'node_id': '15121809f7d670bdd9da8defb356391ee798d55d0f6ec15625582a0d'}

The Ray Dashboard URL is printed above and also part of the output dictionary item `webui_url`

(When using the Anyscale platform, use the URL provided by your instructor to access the Ray Dashboard.)

## Ray Task Dependencies

Let's define a few remote tasks, which will have _dependency_ relationships. We'll learn how Ray handles these dependent, asynchronous computations.

One task will return a random NumPy array of some size `n` and the other task will add two such arrays. We'll also add a sleep time, one tenth the size of `n` to simulate expensive computation.

> **Note:** Dependencies and how Ray implements handling of them are explored in depth in the [03: Ray Internals](03-Ray-Internals.ipynb) lesson. 

In [3]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [4]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [5]:
start = time.time()
ref1 = make_array.remote(20)
ref2 = make_array.remote(20)
ref3 = add_arrays.remote(ref1, ref2)
print(ray.get(ref3))
pd(time.time() - start, prefix="Total time:")

[ 1.43108382  0.70087318 -1.20089303  0.57864184 -0.26180632  1.67105153
  2.35791376  0.83222052  0.54114676 -2.19455132 -0.72782443  2.07478372
  2.86786932  0.21751107 -1.33765011 -2.18729223  1.95132309  2.28626872
 -0.69875823  0.69579854]
Total time: duration:  4.141 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `ref1` and `ref2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural and Pythonic.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, Ray automatically handles task dependencies for us.

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Even though three task invocations occurred, we only used one call to `ray.get()`, when we actually needed the final results. Eliminating unnecessary `ray.get()` calls helps avoid forcing tasks to become synchronous when they could be asynchronous. So, keep these two key points in mind:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

Here is an idiomatic way to use `ray.get()`, where we fire all five asynchronous tasks at once, then ask for all the results at once with `ray.get()`:

In [6]:
start = time.time()

# Comprehension list: five NumPy object references or futures created
array_refs = [make_array.remote(n*10) for n in range(5)]

# Comprehension list: object references or futures of the result of addition
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

# Iterate o er the list of object references or futures
for array in ray.get(added_array_refs):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [-4.07638296 -1.95163267 -0.00880249 -0.29232239  2.42449107  4.19294359
 -0.0503383  -3.55753786  1.89963487 -4.5199296 ]
20: [-1.15085882  1.77579832 -0.01225805 -0.309953    0.21715217 -1.89030721
  4.78175552  1.88517307 -0.18699982  0.29107438 -5.98558067 -0.08797277
 -2.66273081  0.58718528  1.50876663  0.40536929 -2.68862281 -0.91197708
 -1.44423352 -2.39514431]
30: [ 2.56039301 -1.6346322  -0.60871896  1.16193879 -5.70744411  0.19742551
  0.37885034 -1.5600288  -2.45400092  0.48869232  2.2066744   1.03782254
  0.44751701  0.96727377  0.94322784  0.24640674  2.22442813 -0.85789955
 -0.95187082  0.09662156 -2.30043352  0.42470415  2.4074558   2.69538886
  1.83667622 -1.73273107  1.44737938  0.0259227  -0.93103566  2.6686603 ]
40: [ 0.90526841  0.08500723 -4.4759343  -1.69614532  4.00903602  1.70152613
 -3.66922862  0.88682751  1.78039434 -0.21732936 -2.21481583 -0.07879783
 -0.10482116  0.42625183  1.93473718 -0.54609273 -3.21441822 -0.08517154
  1.14747553  2.98695593 

This takes about eight seconds, four seconds for the longest invocation invocation of `make_array`, `make_array(4)`  , and four seconds with longest invocation of `add_arrays`, when passed the results of `make_array(4)`. 

We did the right thing inside each list comprehension. We started the asynchronous tasks all at once and allowed Ray to handle the dependencies. Then we waited on one `ray.get()` call for all the output. 

However, what you see is no output and then everything is suddenly printed at once after eight seconds.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks finish more quickly in our contrived example. We would like to process those results as soon as they are available, even while others continue to run. We'll use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. 

Here is the previous example rewritten to use `ray.wait()`:

In [7]:
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_refs), time.time() - start))
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.004)
0: []
Returned   1 completed tasks. (elapsed time:  2.009)
10: [ 1.9431695  -0.73601346  2.91265181  1.26958173 -0.31414387 -0.58221856
 -2.04464543  2.59826408  2.13087564 -2.56688933]
Returned   1 completed tasks. (elapsed time:  4.014)
20: [ 2.88432484 -3.11871942 -0.04592315  1.33617888 -2.8067684  -0.59656667
 -1.74968928 -0.97351565  0.0107425  -0.32481627 -0.42829614  0.68200873
  1.07150396  1.86634426 -0.61638403 -1.12001991  2.18839802  2.52251977
  0.13937517  1.02194083]
Returned   1 completed tasks. (elapsed time:  6.015)
30: [ 4.54483047  0.6518823   1.03674642  1.94455729 -1.67532543 -1.07640136
 -1.58205622 -1.23547825 -1.35332424  3.60820431 -2.08436084 -2.73765516
 -0.93300154  1.03793909 -0.01560521 -0.00754232  2.29358795 -0.02512471
  1.58556402 -0.40110198  0.27547143 -1.45850163  0.73047698  2.5528473
 -1.5648741   1.79714557  1.34442352 -1.78479468 -0.70858361 -1.64989786]
Returned   1 completed tasks. (elapse

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the refs that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [8]:
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_refs) > 1 else 1
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_refs), time.time() - start))
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.013)
0: []
10: [-2.94487463 -0.20515211 -1.21721959 -2.54505005  2.53539694 -2.89842961
 -2.32593868  0.4861679  -2.57712589 -1.74041328]
Returned   2 completed tasks. (elapsed time:  6.014)
20: [ 0.15756339  0.70655518  0.43356913  0.64729627 -0.78901818 -2.0719179
  0.9722761   0.4325182  -2.26739337 -0.08784976  0.17736519 -3.39512875
  1.5598177  -0.83083243 -3.92696585 -2.60103557 -2.56880192  0.5517638
  0.99855315 -2.16416783]
30: [-0.07768504 -0.70268598  0.92056113 -1.71179513 -1.318743   -5.69315124
 -0.037428   -0.85751717  3.70692497 -1.67805509  0.0671539   2.6207001
  1.23987118  0.92102111  1.77791423  2.30275174  2.73170802 -2.82441125
 -1.81794507 -2.28215284  2.05025238 -2.46189908 -0.11176304 -1.16338214
 -2.80320196  0.91222451 -2.45406968 -0.97807619 -2.76382338  1.9662463 ]
Returned   1 completed tasks. (elapsed time:  8.011)
40: [ 1.176072    1.19516666 -1.41519632  2.40864955  0.26778945  0.69992192
  1.4436109   0

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

 For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

## Exercise 1

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

See the [solutions notebook](solutions/Advanced-Ray-Solutions.ipynb) for a discussion of this exercise and the subsequent exercises.

In [ ]:
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]

arrays = []
waiting_refs = list(added_array_refs)  # Assign a working list to the full list of refs
while len(waiting_refs) > 0:           # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of refs we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_refs) > 1 else 1
    ready_refs, remaining_refs = ray.wait(waiting_refs, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_refs), time.time() - start))
    new_arrays = ray.get(ready_refs)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_refs = remaining_refs  # Reset this list; don't include the completed refs in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. When looping over calls to `ray.wait()` with a list of object refs for running tasks, remove the previously-completed and retrieved objects from the list.
> 3. Don't ask for results you don't need.
> 4. Don't ask for the results you need until you really need them.

## Exercise 3

Let's practice converting a slow loop to Ray, including using `ray.wait()`. Change the function to be a Ray task. Change the invocations to use the `ray.wait()` idiom. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

In [ ]:
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
squares = [slow_square(n) for n in range(4)]
for square in squares:
    print (f'finished: {square}')
duration = time.time() - start

In [ ]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

## Limiting Task Invocations and Retries on Failure

> **Note:** This feature may change in a future version of Ray. See the latest details in the [Ray documentation](https://docs.ray.io/en/latest/package-ref.html#ray.remote). 

Two options you can pass to `ray.remote` when defining a task affect how often it can be invoked and retrying on failure:

* `max_calls`: This specifies the maximum number of times that a given worker can execute the given remote function before it must exit. This can be used to address memory leaks in third-party libraries or to reclaim resources that cannot easily be released, e.g., GPU memory that was acquired by TensorFlow. By default this is infinite.
* `max_retries`: This specifies the maximum number of times that the remote function should be rerun when the worker process executing it crashes unexpectedly. The minimum valid value is 0, the default is 4, and a value of -1 indicates infinite retries are allowed.

Example:

```python
@ray.remote(max_calls=10000, max_retries=10)
def foo():
    pass
```

See the [ray.remote()](https://docs.ray.io/en/latest/package-ref.html#ray.remote) documentation for all the keyword arguments supported.

### Overriding with config()

Remote task and actor objects returned by `@ray.remote` can also be dynamically modified with the same arguments supported by `ray.remote()` using `options()` as in the following examples:

```python
@ray.remote(num_gpus=1, max_calls=1, num_return_vals=2)
def f():
    return 1, 2
g = f.options(num_gpus=2, max_calls=None)
```

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [9]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [10]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [11]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

0.00010609626770019531 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [12]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.2779512405395508 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [13]:
start = time.time()
refs = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(refs)
print(f'{time.time() - start} seconds')

0.1783127784729004 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

## Exercise 4

1. Try incrementing size of n to 2n
2. Do you see a marked difference between the local vs remote times?
3. For the brave, try using `mathplotlib` to plot it.

## Profiling Tasks with ray.timeline()

Sometimes you need to debug performance problems in Ray tasks. Calling `ray.timeline(file)` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.timeline)) captures profiling information for subsequent task execution to the specified file. Afterwards, you can view the data in a Chrome web browser. The format used is unique to Chrome, so Chrome has be used to view the data.

Let's try it with our `make_array` and `add_arrays` methods in the following code. First some potential cleanup:

In [14]:
timeline_file = 'task-timeline.txt' # Will be found in the same directory as this notebook.
if os.path.isfile(timeline_file):   # Delete old one, if an old one exists already.
    os.remove(timeline_file)

In [15]:
ray.timeline(timeline_file)
start = time.time()
array_refs = [make_array.remote(n*10) for n in range(5)]
added_array_refs = [add_arrays.remote(ref, ref) for ref in array_refs]
for array in ray.get(added_array_refs):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [ 0.97332529  1.74874549  2.61013597 -0.63826469  0.5627928  -0.14394584
 -0.80192058  0.96795231 -1.01147608 -0.19958733]
20: [ 0.65613874 -1.80805681 -1.619655   -2.4529704   0.15821344  0.53527084
 -1.24334158  2.21637178  0.46443527 -0.2710335   0.27033077 -1.49718947
  2.80627222  0.7553748  -0.97830888  1.83637038  0.71262775  0.80207359
  1.02618509  0.57165852]
30: [-1.97400065e+00  2.24373538e+00  4.76793568e-01  1.44951452e+00
  2.91712408e+00 -9.72138963e-02 -1.70001976e+00 -7.68350192e-01
 -7.85106961e-01 -2.52866228e+00 -1.36515726e-03 -2.26545083e+00
  9.20154554e-01  1.49150340e+00 -5.54725378e-01 -2.21724933e+00
  1.11076770e+00  1.96036959e+00 -1.81648604e+00 -2.43074034e-01
 -1.01406263e+00  2.92731725e+00 -4.52891339e+00 -1.30303742e+00
 -1.52011057e+00  1.06673214e+00  1.31958021e+00  3.29961399e+00
 -2.27372910e+00 -3.07394294e+00]
40: [-2.82047830e+00  2.00584457e+00 -3.52475321e-01  1.89035438e-01
  4.25828933e+00 -2.47227525e+00  5.63920051e-01 -4.8148

Now, to view the data:

1. Open Chrome and enter chrome://tracing.
2. Click the _Load_ button to load the `task-timeline.txt` file, which will be in this notebook's directory. 
3. To zoom in or out, click the "asymmetric" up-down arrow button. Then hold the mouse button in the graph and roll the mouse scroll wheel up or down. (On a laptop trackpad, press and hold, then use another finger to slide up and down.)
4. To move around, click the crossed arrow and drag a section in view. 
5. Click on a box in the timeline to see details about it. 

Look for blocks corresponding to long-running tasks and look for idle periods, which reflect processing outside the context of Ray.

Here is a screen grab profiling the previous code, zoomed in on one block of tasks and with one task selected. Note the processes shown on the left for drivers (more than one notebook was running at this time) and workers.

![Ray Trace Example](../images/Ray-Trace-Example.png)

In [16]:
ray.shutdown()  # "Undo ray.init()".

The next lesson, [Ray Actors Revisited](02-Ray-Actors-Revisited.ipynb), revisits actors. It provides a more in-depth look at actor characteristics and profiling actor performance using the _Ray Dashboard_.